In [1]:
import torch as th
import numpy as np
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F
from tqdm import tqdm
from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN
from Bayesian.BayesianCNN import BBBAlexNet
from Frequentist.FrequentistCNN import AlexNet

In [5]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
activation_type = 'softplus'  # 'softplus' or 'relu'
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}

n_epochs = 5
lr_start = 0.001
num_workers = 4
valid_size = 0.2
batch_size = 256
train_ens = 1
valid_ens = 1
beta_type = 0.1 

In [4]:
trainset, testset, inputs, outputs = data.getDataset('CIFAR10')
train_loader, valid_loader, test_loader = data.getDataloader(
    trainset, testset, valid_size, batch_size, num_workers)
net = BBBAlexNet(outputs, inputs, priors, activation_type).to(device)
criterion = metrics.ELBO(len(trainset)).to(device)
optimizer = Adam(net.parameters(), lr=lr_start)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
valid_loss_max = np.Inf
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc10_train_loss, bc10_train_acc, bc10_train_kl = BCNN.train_model(net, optimizer, criterion, train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_valid_loss, bc10_valid_acc = BCNN.validate_model(net, criterion, valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    lr_sched.step(valid_loss)

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc10_train_loss, bc10_train_acc, bc10_valid_loss, bc10_valid_acc, bc10_train_kl))

Files already downloaded and verified
Files already downloaded and verified


C:\Users\leona\AppData\Roaming\Python\Python310\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
 20%|████████████████▌                                                                  | 1/5 [03:26<13:44, 206.01s/it]

Epoch: 0 	Training Loss: 38292053.3758 	Training Accuracy: 0.1493 	Validation Loss: 30205720.9000 	Validation Accuracy: 0.1738 	train_kl_div: 376424420.0764


 40%|█████████████████████████████████▏                                                 | 2/5 [07:31<11:27, 229.03s/it]

Epoch: 1 	Training Loss: 25851155.1083 	Training Accuracy: 0.2234 	Validation Loss: 22229799.9000 	Validation Accuracy: 0.2506 	train_kl_div: 257407123.6688


...

Epoch: 2 	Training Loss: 19645173.1083 	Training Accuracy: 0.2647 	Validation Loss: 17365266.9500 	Validation Accuracy: 0.2836 	train_kl_div: 195440729.5796


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [15:02<03:43, 223.74s/it]

Epoch: 3 	Training Loss: 15605968.3121 	Training Accuracy: 0.2819 	Validation Loss: 14020036.7250 	Validation Accuracy: 0.2635 	train_kl_div: 155079582.1656


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [18:17<00:00, 219.51s/it]

Epoch: 4 	Training Loss: 12740687.9108 	Training Accuracy: 0.2958 	Validation Loss: 11573468.8500 	Validation Accuracy: 0.2646 	train_kl_div: 126435828.7389


In [ ]:
trainset, testset, inputs, outputs = data.getDataset('CIFAR100')
train_loader, valid_loader, test_loader = data.getDataloader(
    trainset, testset, valid_size, batch_size, num_workers)
net = BBBAlexNet(outputs, inputs, priors, activation_type).to(device)
criterion = metrics.ELBO(len(trainset)).to(device)
optimizer = Adam(net.parameters(), lr=lr_start)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
valid_loss_max = np.Inf
for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc100_train_loss, bc100_train_acc, bc100_train_kl = BCNN.train_model(net, optimizer, criterion, train_loader, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_valid_loss, bc100_valid_acc = BCNN.validate_model(net, criterion, valid_loader, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    lr_sched.step(valid_loss)

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc100_train_loss, bc100_train_acc, bc100_valid_loss, bc100_valid_acc, bc100_train_kl))

In [ ]:
trainset, testset, inputs, outputs = data.getDataset('CIFAR10')
train_loader, valid_loader, test_loader = data.getDataloader(
    trainset, testset, valid_size, batch_size, num_workers)
net = AlexNet.(outputs, inputs).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=lr)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
valid_loss_min = np.Inf
for epoch in tqdm(range(n_epochs)):

    fc10_train_loss, fc10_train_acc = FCNN.train_model(net, optimizer, criterion, train_loader)
    fc10_valid_loss, fc10_valid_acc = FCNN.validate_model(net, criterion, valid_loader)
    lr_sched.step(valid_loss)
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc10_train_loss, fc10_train_acc, fc10_valid_loss, fc10_valid_acc))

In [ ]:
trainset, testset, inputs, outputs = data.getDataset('CIFAR100')
train_loader, valid_loader, test_loader = data.getDataloader(
    trainset, testset, valid_size, batch_size, num_workers)
net = AlexNet.(outputs, inputs).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=lr)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True)
valid_loss_min = np.Inf
for epoch in tqdm(range(n_epochs)):

    fc100_train_loss, fc100_train_acc = FCNN.train_model(net, optimizer, criterion, train_loader)
    fc100_valid_loss, fc100_valid_acc = FCNN.validate_model(net, criterion, valid_loader)
    lr_sched.step(valid_loss)
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc100_train_loss, fc100_train_acc, fc100_valid_loss, fc100_valid_acc))

In [ ]:
epochs = range(1, 201)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_train_acc, label='bc10')
plt.plot(epochs, bc100_train_acc, label='bc100')
plt.plot(epochs, fc10_train_acc, label='fc10')
plt.plot(epochs, fc100_train_acc, label='fc100')

plt.title('Comparison of training accuracies')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
epochs = range(1, 201)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_valid_acc, label='bc10')
plt.plot(epochs, bc100_valid_acc, label='bc100')
plt.plot(epochs, fc10_valid_acc, label='fc10')
plt.plot(epochs, fc100_valid_acc, label='fc100')

plt.title('Comparison of validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.show()